## Introduction
Objects in JavaScript have an internal property, denoted in the specification as `[[Prototype]]`. When we read a property from an object and it is not present, the `[[Prototype]]` is checked for the presence of the property.

As an example,

In [1]:
let gamepad = {
    model: 'Xbox controller',
    manufacturer: 'Microsoft'
}

gamepad.toString()

[object Object]


We did not define a `toString` property for gamepad object. It is present on gamepad's prototype which can be accessed using `Object.getPrototypeOf` method. Or using non-standard `__proto__` property:

In [ ]:
Object.getPrototypeOf(gamepad) === gamepad.__proto__ // true

`Object.getPrototypeOf(gamepad)` returns an object which contains multiple properties:

```
{
    ...,
    __proto__: null,
    constructor: function Object(),
    hasOwnProperty: function hasOwnProperty(),
    isPrototypeOf: function isPrototypeOf(),
    propertyIsEnumerable: function propertyIsEnumerable(),
    toLocaleString: function toLocaleString(),
    toString: function toString(),
    valueOf: function valueOf(),
    ...
}
```

This object is a property of `Object` constructor called `prototype` (every function has this property. Discussed later). To set the prototype to a different object, use `Object.setPrototypeOf` (or set `__proto__`)

In [6]:
let animal = {
    age: 0
};

let horse = {
    breed: 'Arabian'
};

Object.setPrototypeOf(horse, animal);
horse.age;

0


What happens when we set `age` property of `horse`?

In [ ]:
horse.age = 5;

console.log(horse.age);  // 5
console.log(animal.age); // 0

`horse` now has its own property `age` separate from `age` property of `animal`. This essentially means that you cannot alter the state of prototype.

## `F.prototype`
Every function has a property `prototype` associated with. When the function is used as a constructor (using `new`) the newly created object's prototype is set to the `prototype` property of the function.

In [ ]:
// Arrays
let array = [];
console.log(array.__proto__ == Array.prototype);

// Number
let number = 5;
console.log(number.__proto__ == Number.prototype);

// Map
let map = new Map();
console.log(map.__proto__ == Map.prototype);

// Function
let fn = function() {}
console.log(fn.__proto__ == Function.prototype);

// Object
let object = {}
console.log(object.__proto__ == Object.prototype);

Quickly looking at the prototypes:
- **Array.prototype:** is an array having the following properties defined:
  ```
    at: function at(),
    concat: function concat(),
    constructor: function Array(),
    copyWithin: function copyWithin(),
    entries: function entries(),
    every: function every(),
    fill: function fill(),
    filter: function filter(),
    length: 0,
    ...
  ```
- **Number.prototype:** is an object containing number of functions. It defines its own custom `toString` and `valueOf`.
  ```
    constructor: function Number(),
    toExponential: function toExponential(),
    toFixed: function toFixed(),
    toLocaleString: function toLocaleString(),
    toPrecision: function toPrecision(),
    toString: function toString(),
    valueOf: function valueOf(),
    ...
  ```
- **Function.prototype:** contains methods like `bind`, `apply`, `call`, etc.
- **Object.prototype:** is an object containing:
  ```
    ...
    hasOwnProperty: function hasOwnProperty(),
    isPrototypeOf: function isPrototypeOf(),
    propertyIsEnumerable: function propertyIsEnumerable(),
    toLocaleString: function toLocaleString(),
    toString: function toString(),
    valueOf: function valueOf(),
    ...
  ```

**Constructor call:** in JavaScript we do not have a constructor function, rather we have a constructor call. Whenever we call a function with `new` keyword, we do a constructor call. A sequence of actions occur when we use `new`:
- a new blank object is created
- this new object's internal `[[Prototype]]` is linked to the "constructor" function's `prototype` property.
- any `this` keyword used in the "constructor" function now refers to the newly created object.
- if the function doesn't return anything, this new object is returned.

In [ ]:
function Box(value) {
    this.value = value;
    return this;
}

let boxProto = {
    print() {
        console.log(this.value);
    }
};

Box.prototype = boxProto;

let box = new Box("a bunch of words");
box.print(); // a bunch of words
console.log(Object.getPrototypeOf(box) == boxProto);  // true

Within the context of a function it is possible to know whether it was called using `new` operator or not. Introduced in ES6, the special meta-property `new.target` tells us whether the function (or constructor) was called using `new`.

In [ ]:
function Person(name) {
  if (!new.target) {  // new.target should refer to the constructor function
    throw new Error("Person() must be called with new");
  }
  this.name = name;
}

### Default Prototype
The default `prototype` property of a function is an object containing one property `constructor` which points to the same function.

In [ ]:
function User(age) {
    this.age = age;
    return this;
}

/* default prototype
User.prototype = { constructor: User };
*/

Typically we add items to the default prototype, like:

In [ ]:
User.prototype.isAdult = function() {
    return this.age >= 18;
}

## Prototype Chain

Consider the example below:

In [ ]:
function Employee(id, name){
    this.id = id
    this.name = name
}

Employee.prototype.identify = function(){
    return `Id: ${this.id}, Name: ${this.name}`
}

function Manager(id, name, level){
    Employee.call(this, id, name)
    this.level = level
}

Manager.prototype = Object.create(Employee.prototype)

Manager.prototype.setDirects = function(list){
    this.directs = list
}

let m1 = new Manager(1, 'John Doe', 3)
let m2 = new Manager(2, 'Jane Doe', 4)

console.log(m1.identify()) // Id: 1, Name: John Doe
console.log(m2.identify()) // Id: 1, Name: John Doe

This creates a chain like:

<img src="./images/prototype_chain.png" width=750 height=auto />

## Class
A simple class looks like:

In [1]:
class Person {
    constructor(name, age) {
        this.name = name;
        this.age = age;
    }

    print() {
        console.log(`Person name is ${this.name} and age is ${this.age}`);
    }
}

const p1 = new Person("John Doe", 32);
p1.print();

Person name is John Doe and age is 32


The above `class` definition is equivalent to:

In [ ]:
function Person(name, age) {
    this.name = name;
    this.age = age;
}

Person.prototype = {
    print() {
        console.log(`Person name is ${this.name} and age is ${this.age}`);
    },
    constructor: Person
}

// Or just Person.prototype.print = ...

Which we can verify by:

In [2]:
console.log(typeof Person);
console.log(Person.prototype.constructor === Person);
console.log(Person.prototype.print);

function
true
[Function: print]


Constructor function defined above is not exactly how class works, there are certain differences:
- you cannot call `Person` without `new`: 

In [3]:
Person("Mad Max", 35);

1:1 - Value of type 'typeof Person' is not callable. Did you mean to include 'new'?


- class methods are non-enumerable by default

In [4]:
Object.getOwnPropertyDescriptor(Person.prototype, "print")

{
  value: [Function: print],
  writable: true,
  enumerable: false,
  configurable: true
}


### Class Expression
Similar to function expressions, we also have class expressions:

In [5]:
let User = class {
  constructor(username, password) {
      this.username = username;
      this.password = password;
  }  
};

console.log(new User("admin", "pa$$w0rd").password);

pa$$w0rd


### Getters and Setters

In [6]:
class Coordinate {
    constructor(x, y) {
        this._x = x;
        this._y = y;
    }

    get x() { return this._x; }
    set x(x) { this._x = x; }

    get y() { return this._y; }
    set y(y) { this._y = y; }
}

let origin = new Coordinate(0, 0);
console.log(`Origin has x=${origin.x} and y=${origin.y}`);

Origin has x=0 and y=0


### Computed Name

In [ ]:
class Sample {
    ["print" + "Line"](text) {
        console.log(text);
    }
}

### Class Fields
Allow us to add properties:

In [7]:
class List {
    store = [];
    capacity = 0;
    occupied = 0;
}

let list = new List();
list.occupied;

0


### Static Members
Use the `static` keyword to defined static variables and methods:

In [8]:
class Product {
    static specVersion = "1.0";

    constructor(name, price) {
        this.name = name;
        this.price = price;
    }

    static equals(p1, p2) {
        return p1.name === p2.name && p1.price === p2.price;
    }
}

let p1 = new Product("Apple Airbuds", 299);
let p2 = new Product("Apple Airbuds", 299);
console.log(Product.equals(p1, p2));

// Cannot do p1.equals() or p1.specVersion. These are only accessible through Product

true


This is equivalent to:

In [ ]:
function Product(name, price) {
    this.name = name;
    this.price = price;
}

// Static property
Product.specVersion = "1.0";

// Static method
Product.equals = function(p1, p2) {
    return p1.name === p2.name && p1.price === p2.price;
};

### Class Inheritance
Use the `extend` keyword:

In [9]:
class UIElement {
    constructor(id) {
        this.id = id;
    }
    
    size() {
        console.log(`Size of element ${this.id} is being computed`);
    }
}

class Button extends UIElement {
    click() {
        console.log(`Clicked button with id ${this.id}`);
    }
}

let submitButton = new Button(5);
submitButton.click();

Clicked button with id 5


This is equivalent to writing:

In [ ]:
function UIElement(id) {
    this.id = id;
}
UIElement.prototype.size = function() {
    console.log(`Size of element ${this.id} is being computed`);
}

function Button(id) {
    UIElement.call(this, id);
}
Button.prototype = Object.create(UIElement.prototype);
Button.prototype.constructor = Button;
Button.prototype.click = function() {
    console.log(`Clicked button with id ${this.id}`);
}

`static` members are not inherited as they are member of the constructor function and not its `prototype`.

When constructor is missing from class definition:
- an empty constructor is automatically created when class does not extend another class
- when class extends another class, then the automatically created class looks like:

In [ ]:
class Child extends Parent {
  constructor(...args) {
    super(...args);
  }
}

This is a derived constructor which affects its behavior with `new`:
- when a regular function is executed with `new`, it creates an empty object and assigns it to this.
- but when a derived constructor runs, it doesn’t do this. It expects the parent constructor to do this job.

Which means call to `super` must be the first statement.

**`super` keyword:** can be used to call the parent class' constructor or method.

In [ ]:
class Input extends UIElement {
    constructor(id, width) {
        super(id);          // Derived constructor, therefore first call must be to super
        this.width = width;
    }

    size() {
        super.size();
        console.log(`Size is ${this.width}`);
    }
}

Equivalent to:

In [ ]:
function UIElement(id) {
    this.id = id;
}
UIElement.prototype.size = function() {
    console.log(`Size of element ${this.id} is being computed`);
}

function Input(id) {
    UIElement.call(this, id);
    this.width = width;
}
Input.prototype = Object.create(UIElement.prototype);
Input.prototype.constructor = Button;
Input.prototype.click = function() {
    UIElement.prototype.size.call(this);
    console.log(`Clicked button with id ${this.id}`);
}

This means that both the constructor calls below return the same output:

In [10]:
class Animal {
    name = "animal"

    constructor() {
        console.log(this.name);
    }
}

class Horse extends Animal {
    name = "horse"
}

let animal = new Animal();
let horse = new Horse();  // horse.name did not exist when the constructor was called

animal
animal


Equivalent to:

In [ ]:
function Animal() {
  this.name = 'animal';
  alert(this.name);
}

function Rabbit() {
  Animal.call(this);   // runs Animal() in this instance context
  this.name = 'rabbit';
}

Rabbit.prototype = Object.create(Animal.prototype);
Rabbit.prototype.constructor = Rabbit;

When we change this to:

In [11]:
class Animal {
  showName() {  // instead of this.name = 'animal'
    console.log('animal');
  }

  constructor() {
    this.showName(); // instead of alert(this.name);
  }
}

class Horse extends Animal {
  showName() {
    console.log('horse');
  }
}

new Animal();
new Horse();

animal
horse
Horse {}


UncaughtException: Error: Unexpected pending rebuildTimer
    at sys.setTimeout (C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\dist\converter.js:111:19)
    at scheduleProgramUpdate (C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122735:35)
    at onSourceFileChange (C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122876:7)
    at C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:122868:56
    at cb (C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\dist\converter.js:184:13)
    at C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5798:9
    at C:\Users\salman\AppData\Roaming\npm\node_modules\tslab\node_modules\@tslab\typescript-for-tslab\lib\typescript.js:5560:101
    at Array.forEach (<anonymous>)
    at FSWatcher

This is because:

In [ ]:
function Animal() {
  this.showName();
}
Animal.prototype.showName = function() {
    console.log("animal");
}

function Horse() {
  Animal.call(this);  // --> this is reference to horse object
}
Horse.prototype = Object.create(Animal.prototype);
Horse.prototype.constructor = Horse;
Horse.prototype.showName = function() {
    console.log("horse");
}